In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem.SaltRemover import SaltRemover
from rdkit.Chem import inchi as rd_inchi
from chembl_structure_pipeline import standardizer
from rdkit.Chem import rdMolDescriptors
import os

# Constantes
VARIANCE_THRESHOLD = 0.2  # Threshold para variação de log em problemas de regressão

# Instanciação do SaltRemover uma única vez
remover = SaltRemover()

def remove_salts(smile):
    """Processa a molécula, removendo sais e verificando validade."""
    try:
        mol = Chem.MolFromSmiles(smile)
        if mol is None:
            return None
        # Utiliza a instância 'remover' para remover sais
        cleaned_mol = remover.StripMol(mol, dontRemoveEverything=True)
        if cleaned_mol.GetNumAtoms() > 2:
            return Chem.MolToSmiles(cleaned_mol, kekuleSmiles=True)
    except Exception as e:
        print(f"Erro ao processar SMILES '{smile}': {e}")
        return None
    return None

def remove_salts_stage(df, smiles_col):
    """Remove sais e moléculas inválidas, atualizando a coluna de SMILES final."""
    print("Removendo sais das moléculas...")
    
    # Aplicar a função de remoção de sais e armazenar em uma coluna temporária
    df['final_smiles_cleaned'] = df[smiles_col].apply(remove_salts)
    
    # Identificar os compostos removidos (onde a remoção de sais resultou em None)
    removed_salts_df = df[df['final_smiles_cleaned'].isna()].copy()
    removed = removed_salts_df.shape[0]
    
    # Atualizar a coluna 'final_smiles' com os SMILES limpos
    df['final_smiles'] = df['final_smiles_cleaned']
    
    # Remover a coluna temporária
    df = df.drop(columns=['final_smiles_cleaned'])
    
    # Remover os compostos que não possuem SMILES válidos após a remoção de sais
    df = df.dropna(subset=['final_smiles']).reset_index(drop=True)
    
    print(f"Removidos {removed} compostos durante a remoção de sais.")
    return df, removed, removed_salts_df

def remove_mixtures_stage(df):
    """Remove misturas (moléculas com pontos), retornando os compostos removidos."""
    print("Removendo misturas de moléculas...")
    initial_count = df.shape[0]
    removed_mixtures = df[df['final_smiles'].str.contains(r'\.')]
    removed = removed_mixtures.shape[0]
    df = df[~df['final_smiles'].str.contains(r'\.')].reset_index(drop=True)
    print(f"Removidos {removed} compostos que são misturas.")
    return df, removed, removed_mixtures

def standardize_smiles_stage(df, smiles_col):
    """Padroniza os SMILES."""
    print("Padronizando os SMILES...")
    def standardize_smiles(s):
        try:
            mol = Chem.MolFromSmiles(s, sanitize=True)
            if mol is None:
                return None
            mol_block = Chem.MolToMolBlock(mol)
            std_mol_block = standardizer.standardize_molblock(mol_block)
            std_mol = Chem.MolFromMolBlock(std_mol_block)
            return Chem.MolToSmiles(std_mol, kekuleSmiles=True)
        except Exception as e:
            print(f"Erro ao padronizar SMILES '{s}': {e}")
            return None

    df['final_smiles'] = df[smiles_col].apply(standardize_smiles)
    removed = df['final_smiles'].isna().sum()
    df = df.dropna(subset=['final_smiles']).reset_index(drop=True)
    print(f"Removidos {removed} compostos durante a padronização dos SMILES.")
    return df, removed

def calculate_inchi_stage(df):
    """Calcula InChI para identificação de duplicatas."""
    print("Calculando InChI para as moléculas...")
    def mol_to_inchi(s):
        try:
            mol = Chem.MolFromSmiles(s)
            if mol:
                return Chem.inchi.MolToInchi(mol)
            else:
                return None
        except Exception as e:
            print(f"Erro ao calcular InChI para SMILES '{s}': {e}")
            return None

    df['InChI'] = df['final_smiles'].apply(mol_to_inchi)
    removed = df['InChI'].isna().sum()
    df = df.dropna(subset=['InChI']).reset_index(drop=True)
    print(f"Removidos {removed} compostos devido a falhas no cálculo do InChI.")
    return df, removed

def remove_duplicates_regression(df, target_col, threshold=VARIANCE_THRESHOLD, save_log_target=True):
    """
    Remove duplicatas em datasets de regressão baseado na variação logarítmica:
    - Duplicatas Concordantes: Mantém um exemplar com Outcome igual à média dos valores originais.
    - Duplicatas Discordantes: Remove todas as duplicatas.

    Parâmetros:
    - df (pd.DataFrame): DataFrame contendo os dados.
    - target_col (str): Nome da coluna contendo o valor de regressão (outcome).
    - threshold (float): Valor limite para a variação logarítmica.
    - save_log_target (bool): Se True, mantém os valores logarítmicos (log_target) no DataFrame final.

    Retorna:
    - pd.DataFrame: DataFrame filtrado, incluindo opcionalmente os valores logarítmicos.
    - int: Número de duplicatas concordantes removidas.
    - int: Número de duplicatas discordantes removidas.
    - pd.DataFrame: DataFrame com duplicatas concordantes removidas.
    - pd.DataFrame: DataFrame com duplicatas discordantes removidas.
    """
    print("Removendo duplicatas para tarefas de regressão...")
    
    # Verificar se a coluna contém apenas valores numéricos
    df[target_col] = pd.to_numeric(df[target_col], errors='coerce')
    
    # Remover valores não numéricos
    invalid_values = df[target_col].isna().sum()
    if invalid_values > 0:
        print(f"Atenção: {invalid_values} valores não numéricos encontrados em '{target_col}' e serão removidos.")
        df = df.dropna(subset=[target_col]).reset_index(drop=True)
    
    # Remover valores não positivos (necessário para cálculo de log)
    non_positive_values = (df[target_col] <= 0).sum()
    if non_positive_values > 0:
        print(f"Atenção: {non_positive_values} valores não positivos encontrados em '{target_col}' e serão removidos.")
        df = df[df[target_col] > 0].reset_index(drop=True)
    
    # Calcular o logaritmo do target
    df['log_target'] = np.log(df[target_col])
    
    # Agrupar pelo InChI e processar duplicatas
    grouped = df.groupby('InChI')
    concordant_removed = 0
    discordant_removed = 0
    keep_rows = []
    removed_concordant_rows = []  # Lista para armazenar os concordantes removidos
    removed_discordant_rows = []  # Lista para armazenar os discordantes removidos

    for inchi, group in grouped:
        if len(group) > 1:
            std = group['log_target'].std()
            if std <= threshold:
                # Duplicatas concordantes: calcular média dos Outcomes
                mean_outcome = group[target_col].mean()
                concordant_removed += (len(group) - 1)
                keep = group.iloc[0].copy()
                keep[target_col] = mean_outcome
                keep_rows.append(keep)
                # Remover as demais entradas
                removed = group.iloc[1:]
                removed_concordant_rows.append(removed)
            else:
                # Duplicatas discordantes: remover todas
                discordant_removed += len(group)
                removed_discordant_rows.append(group)
        else:
            keep_rows.append(group.iloc[0])

    # Criar DataFrame filtrado com ou sem log_target
    if keep_rows:
        filtered_df = pd.DataFrame(keep_rows).reset_index(drop=True)
    else:
        filtered_df = pd.DataFrame(columns=df.columns)

    if not save_log_target:
        filtered_df = filtered_df.drop(columns=['log_target'])

    # Concatenar removidos
    if removed_concordant_rows:
        removed_concordant_df = pd.concat(removed_concordant_rows, ignore_index=True)
    else:
        removed_concordant_df = pd.DataFrame(columns=df.columns)

    if removed_discordant_rows:
        removed_discordant_df = pd.concat(removed_discordant_rows, ignore_index=True)
    else:
        removed_discordant_df = pd.DataFrame(columns=df.columns)

    print(f"Duplicatas concordantes removidas: {concordant_removed}")
    print(f"Duplicatas discordantes removidas: {discordant_removed}")
    return filtered_df, concordant_removed, discordant_removed, removed_concordant_df, removed_discordant_df

def remove_duplicates_classification(df, outcome_col):
    """
    Remove duplicatas em datasets de classificação:
    - Duplicatas Concordantes: Mantém um exemplar, remove os demais.
    - Duplicatas Discordantes: Remove todas as duplicatas.

    Retorna:
    - DataFrame filtrado
    - Número de duplicatas concordantes removidas
    - Número de duplicatas discordantes removidas
    - DataFrame com as duplicatas concordantes removidas
    - DataFrame com as duplicatas discordantes removidas
    """
    print("Removendo duplicatas para tarefas de classificação...")
    grouped = df.groupby('InChI')
    concordant_removed = 0
    discordant_removed = 0
    keep_indices = []
    removed_concordant_rows = []  # Lista para armazenar os concordantes removidos
    removed_discordant_rows = []  # Lista para armazenar os discordantes removidos

    for inchi, group in grouped:
        if len(group) > 1:
            # Verificar se todos os Outcomes são iguais
            outcomes = group[outcome_col].unique()
            if len(outcomes) == 1:
                # Duplicatas concordantes: manter apenas um
                concordant_removed += (len(group) - 1)
                keep_indices.append(group.index[0])
                removed = group.iloc[1:]
                removed_concordant_rows.append(removed)
            else:
                # Duplicatas discordantes: remover todas
                discordant_removed += len(group)
                removed_discordant_rows.append(group)
        else:
            keep_indices.append(group.index[0])

    final_drop_dup = df.loc[keep_indices].reset_index(drop=True)

    # Concatenar removidos
    if removed_concordant_rows:
        removed_concordant_df = pd.concat(removed_concordant_rows, ignore_index=True)
    else:
        removed_concordant_df = pd.DataFrame(columns=df.columns)

    if removed_discordant_rows:
        removed_discordant_df = pd.concat(removed_discordant_rows, ignore_index=True)
    else:
        removed_discordant_df = pd.DataFrame(columns=df.columns)

    print(f"Duplicatas concordantes removidas: {concordant_removed}")
    print(f"Duplicatas discordantes removidas: {discordant_removed}")
    return final_drop_dup, concordant_removed, discordant_removed, removed_concordant_df, removed_discordant_df


    """
    Remove duplicatas em datasets de regressão baseado na variação logarítmica:
    - Duplicatas Concordantes: Mantém um exemplar com Outcome igual à média dos valores originais.
    - Duplicatas Discordantes: Remove todas as duplicatas.

    Retorna:
    - DataFrame filtrado com médias para duplicatas concordantes
    - Número de duplicatas concordantes removidas
    - Número de duplicatas discordantes removidas
    - DataFrame com as duplicatas concordantes removidas
    - DataFrame com as duplicatas discordantes removidas
    """
    print("Removendo duplicatas para tarefas de regressão...")
    
    # Verificar se a coluna contém apenas valores numéricos
    df[target_col] = pd.to_numeric(df[target_col], errors='coerce')
    
    # Remover valores não numéricos
    invalid_values = df[target_col].isna().sum()
    if invalid_values > 0:
        print(f"Atenção: {invalid_values} valores não numéricos encontrados em '{target_col}' e serão removidos.")
        df = df.dropna(subset=[target_col]).reset_index(drop=True)
    
    # Remover valores não positivos (necessário para cálculo de log)
    non_positive_values = (df[target_col] <= 0).sum()
    if non_positive_values > 0:
        print(f"Atenção: {non_positive_values} valores não positivos encontrados em '{target_col}' e serão removidos.")
        df = df[df[target_col] > 0].reset_index(drop=True)
    
    # Calcular o logaritmo do target
    df['log_target'] = np.log(df[target_col])
    
    # Agrupar pelo InChI e processar duplicatas
    grouped = df.groupby('InChI')
    concordant_removed = 0
    discordant_removed = 0
    keep_rows = []
    removed_concordant_rows = []  # Lista para armazenar os concordantes removidos
    removed_discordant_rows = []  # Lista para armazenar os discordantes removidos

    for inchi, group in grouped:
        if len(group) > 1:
            std = group['log_target'].std()
            if std <= threshold:
                # Duplicatas concordantes: calcular média dos Outcomes
                mean_outcome = group[target_col].mean()
                concordant_removed += (len(group) - 1)
                keep = group.iloc[0].copy()
                keep[target_col] = mean_outcome
                keep_rows.append(keep)
                # Remover as demais entradas
                removed = group.iloc[1:]
                removed_concordant_rows.append(removed)
            else:
                # Duplicatas discordantes: remover todas
                discordant_removed += len(group)
                removed_discordant_rows.append(group)
        else:
            keep_rows.append(group.iloc[0])

    # Criar DataFrame filtrado
    if keep_rows:
        filtered_df = pd.DataFrame(keep_rows).drop(columns=['log_target']).reset_index(drop=True)
    else:
        filtered_df = pd.DataFrame(columns=df.columns.drop('log_target'))

    # Concatenar removidos
    if removed_concordant_rows:
        removed_concordant_df = pd.concat(removed_concordant_rows, ignore_index=True)
    else:
        removed_concordant_df = pd.DataFrame(columns=df.columns)

    if removed_discordant_rows:
        removed_discordant_df = pd.concat(removed_discordant_rows, ignore_index=True)
    else:
        removed_discordant_df = pd.DataFrame(columns=df.columns)

    print(f"Duplicatas concordantes removidas: {concordant_removed}")
    print(f"Duplicatas discordantes removidas: {discordant_removed}")
    return filtered_df, concordant_removed, discordant_removed, removed_concordant_df, removed_discordant_df

def write_log(savepath, log_data):
    """Escreve o log do processo de curadoria."""
    try:
        log_path = os.path.join(savepath, 'curation_log.txt')
        print(f"Salvando log em '{log_path}'...")
        with open(log_path, 'w') as log_file:
            log_file.write('Log do processo de curadoria:\n')
            for key, value in log_data.items():
                log_file.write(f'{key}: {value}\n')
        print("Log salvo com sucesso.")
    except Exception as e:
        print(f"Erro ao salvar o log: {e}")

def curate_dataset(df, smiles_col, outcome_col, task_type='classification', savepath='curated_data'):
    """
    Função principal de curadoria que executa as etapas fixas de curadoria:
    - Remove NaNs de Outcome e SMILES
    - Prepara os SMILES removendo estereoquímica e valida
    - Padroniza estruturalmente os SMILES
    - Remove sais da molécula
    - Remove misturas da molécula
    - Remove duplicatas conforme o tipo de tarefa (classificação ou regressão)

    Parâmetros:
    - df (pd.DataFrame): DataFrame contendo os dados.
    - smiles_col (str): Nome da coluna que contém os SMILES das moléculas.
    - outcome_col (str): Nome da coluna que contém os resultados (Outcome).
    - task_type (str): Tipo de tarefa, 'classification' ou 'regression'. Padrão é 'classification'.
    - savepath (str): Diretório onde os arquivos curados e logs serão salvos. Padrão é 'curated_data'.

    Retorna:
    - pd.DataFrame: DataFrame curado.
    - dict: Dicionário contendo informações do log.
    """
    try:
        # Inicialização do log
        log_data = {}

        # Certificar que o diretório de salvamento existe
        if not os.path.exists(savepath):
            print(f"Criando diretório de salvamento em '{savepath}'...")
            os.makedirs(savepath, exist_ok=True)
            print("Diretório criado.")

        # Capturar quantidade inicial de compostos
        initial_count = df.shape[0]
        log_data['Initial Compounds'] = initial_count
        print(f"Quantidade inicial de compostos: {initial_count}")

        # Remover valores nulos em Outcome e SMILES
        print("Removendo valores nulos em Outcome e SMILES...")
        df = df.dropna(subset=[smiles_col, outcome_col]).reset_index(drop=True)
        removed_after_nan = initial_count - df.shape[0]
        log_data['Removed After Drop NaN Molecule/Outcome'] = removed_after_nan
        print(f"Removidos {removed_after_nan} compostos devido a NaNs.")

        # Preparar os SMILES removendo estereoquímica e validando
        print("Preparando os SMILES removendo estereoquímica e validando...")
        def prepare_smiles(smile):
            """Remove estereoquímica e valida o SMILES."""
            try:
                cleaned_smile = smile.replace('@', '').replace('/', '').replace('\\', '')
                mol = Chem.MolFromSmiles(cleaned_smile)
                if mol:
                    return Chem.MolToSmiles(mol, kekuleSmiles=True)
                else:
                    return None
            except Exception as e:
                print(f"Erro ao preparar SMILES '{smile}': {e}")
                return None

        df['final_smiles'] = df[smiles_col].apply(prepare_smiles)
        removed_invalid_smiles = df['final_smiles'].isna().sum()
        df = df.dropna(subset=['final_smiles']).reset_index(drop=True)
        log_data['Invalid SMILES Removed During Preparation'] = removed_invalid_smiles
        print(f"Removidos {removed_invalid_smiles} compostos com SMILES inválidos durante a preparação.")
        print(f"Quantidade de compostos após preparação dos SMILES: {df.shape[0]}")

        # Padronizar SMILES
        df, removed = standardize_smiles_stage(df, 'final_smiles')
        log_data['SMILES Standardized Removed'] = removed
        print(f"Quantidade de compostos após padronização dos SMILES: {df.shape[0]}")

        # Remover sais
        df, removed_salts, removed_salts_df = remove_salts_stage(df, 'final_smiles')
        log_data['Salts Removed'] = removed_salts
        print(f"Quantidade de compostos após remoção de sais: {df.shape[0]}")

        # Salvar sais removidos
        if not removed_salts_df.empty:
            removed_salts_csv = os.path.join(savepath, 'removed_salts.csv')
            print(f"Salvando sais removidos em '{removed_salts_csv}'...")
            removed_salts_df.to_csv(removed_salts_csv, sep=',', header=True, index=False)
            print("Sais removidos salvos com sucesso.")
        else:
            print("Nenhum sal removido para salvar.")

        # Remover misturas
        df, removed_mixtures, removed_mixtures_df = remove_mixtures_stage(df)
        log_data['Mixtures Removed'] = removed_mixtures
        print(f"Quantidade de compostos após remoção de misturas: {df.shape[0]}")

        # Salvar misturas removidas
        if not removed_mixtures_df.empty:
            removed_mixtures_csv = os.path.join(savepath, 'removed_mixtures.csv')
            print(f"Salvando misturas removidas em '{removed_mixtures_csv}'...")
            removed_mixtures_df.to_csv(removed_mixtures_csv, sep=',', header=True, index=False)
            print("Misturas removidas salvas com sucesso.")
        else:
            print("Nenhuma mistura removida para salvar.")

        # Calcular InChI
        df, removed = calculate_inchi_stage(df)
        log_data['InChI Calculation Removed'] = removed
        print(f"Quantidade de compostos após cálculo de InChI: {df.shape[0]}")

        # Remover duplicatas conforme o tipo de tarefa
        if task_type == 'classification':
            df, concordant, discordant, removed_concordant_dup, removed_discordant_dup = remove_duplicates_classification(df, outcome_col)
            log_data['Concordant Duplicates Removed'] = concordant
            log_data['Discordant Duplicates Removed'] = discordant
            print(f"Quantidade de compostos após remoção de duplicatas (classificação): {df.shape[0]}")
        elif task_type == 'regression':
            df, concordant, discordant, removed_concordant_dup, removed_discordant_dup = remove_duplicates_regression(df, outcome_col)
            log_data['Concordant Duplicates Removed'] = concordant
            log_data['Discordant Duplicates Removed'] = discordant
            print(f"Quantidade de compostos após remoção de duplicatas (regressão): {df.shape[0]}")
        else:
            raise ValueError("task_type deve ser 'classification' ou 'regression'.")

        # Capturar quantidade final de compostos
        final_count = df.shape[0]
        log_data['Final Compounds'] = final_count
        print(f"Quantidade final de compostos: {final_count}")

        # Salvar dataset final
        output_csv = os.path.join(savepath, 'curated_dataset.csv')
        print(f"Salvando dataset curado em '{output_csv}'...")
        df.to_csv(output_csv, sep=',', header=True, index=False)
        print("Dataset curado salvo com sucesso.")

        # Salvar os duplicatas removidos separadamente
        if task_type == 'classification':
            # Salvar duplicatas concordantes removidas
            if not removed_concordant_dup.empty:
                removed_concordant_csv = os.path.join(savepath, 'removed_concordant_duplicates.csv')
                print(f"Salvando duplicatas concordantes removidas em '{removed_concordant_csv}'...")
                removed_concordant_dup.to_csv(removed_concordant_csv, sep=',', header=True, index=False)
                print("Duplicatas concordantes removidas salvas com sucesso.")
            else:
                print("Nenhuma duplicata concordante removida para salvar.")

            # Salvar duplicatas discordantes removidas
            if not removed_discordant_dup.empty:
                removed_discordant_csv = os.path.join(savepath, 'removed_discordant_duplicates.csv')
                print(f"Salvando duplicatas discordantes removidas em '{removed_discordant_csv}'...")
                removed_discordant_dup.to_csv(removed_discordant_csv, sep=',', header=True, index=False)
                print("Duplicatas discordantes removidas salvas com sucesso.")
            else:
                print("Nenhuma duplicata discordante removida para salvar.")

        elif task_type == 'regression':
            # Salvar duplicatas concordantes removidas
            if not removed_concordant_dup.empty:
                removed_concordant_csv = os.path.join(savepath, 'removed_concordant_duplicates.csv')
                print(f"Salvando duplicatas concordantes removidas em '{removed_concordant_csv}'...")
                removed_concordant_dup.to_csv(removed_concordant_csv, sep=',', header=True, index=False)
                print("Duplicatas concordantes removidas salvas com sucesso.")
            else:
                print("Nenhuma duplicata concordante removida para salvar.")

            # Salvar duplicatas discordantes removidas
            if not removed_discordant_dup.empty:
                removed_discordant_csv = os.path.join(savepath, 'removed_discordant_duplicates.csv')
                print(f"Salvando duplicatas discordantes removidas em '{removed_discordant_csv}'...")
                removed_discordant_dup.to_csv(removed_discordant_csv, sep=',', header=True, index=False)
                print("Duplicatas discordantes removidas salvas com sucesso.")
            else:
                print("Nenhuma duplicata discordante removida para salvar.")

        # Escrever o log
        write_log(savepath, log_data)

        # Retornar dataframe e log
        return df, log_data
    except Exception as e:
        print(f'Erro durante a curadoria do dataset: {e}')
        return None, None

[14:58:45] Initializing Normalizer


In [2]:
# Carregar o arquivo Excel ou CSV
df = pd.read_csv('odorant_odorless.csv')
# ou se for CSV:
#df = pd.read_csv('/EC3LLNA ready.csv')

df  # Para verificar o início do dataset


,SMILES,Label
0,C(C(C1C(=C(C(=O)O1)O)O)O)O,1
1,C(C(C(=O)O)N)S,1
2,CC1=C(SC=[N+]1CC2=CN=C(N=C2N)C)CCO.Cl.[Cl-],1
3,CC(C)(C)O,1
4,CCOCC,1
...,...,...
5088,[Al],0
5089,[Tl],0
5090,CN1C=NC2=C1C(=O)N(C(=O)N2C)C,0
5091,[As],0


In [3]:
curated_df, log = curate_dataset(
    df, 
    smiles_col='SMILES',        # Substitua pelo nome da sua coluna de SMILES
    outcome_col='Label',               # Substitua pelo nome da sua coluna de Outcome
    task_type='classification',               # Ou 'classification' conforme sua necessidade
    savepath='/Users/francisco/Library/CloudStorage/OneDrive-Pessoal/Documentos/LabMol/SOFIA/MQ/Paper OdorSight/DATA/Curation'  # Diretório correto
)

Quantidade inicial de compostos: 5093
Removendo valores nulos em Outcome e SMILES...
Removidos 3 compostos devido a NaNs.
Preparando os SMILES removendo estereoquímica e validando...


[15:00:07] SMILES Parse Error: syntax error while parsing: vCC(C)C(=O)C(=O)[O-].[Na+]
[15:00:07] SMILES Parse Error: check for mistakes around position 1:
[15:00:07] vCC(C)C(=O)C(=O)[O-].[Na+]
[15:00:07] ^
[15:00:07] SMILES Parse Error: Failed parsing SMILES 'vCC(C)C(=O)C(=O)[O-].[Na+]' for input: 'vCC(C)C(=O)C(=O)[O-].[Na+]'
[15:00:07] SMILES Parse Error: Failed parsing SMILES 'OGJATLSJIMPQBD-UHFFFAOYSA-N' for input: 'OGJATLSJIMPQBD-UHFFFAOYSA-N'
[15:00:07] SMILES Parse Error: syntax error while parsing: ZDQWESQEGGJUCH-UHFFFAOYSA-N
[15:00:07] SMILES Parse Error: check for mistakes around position 1:
[15:00:07] ZDQWESQEGGJUCH-UHFFFAOYSA-N
[15:00:07] ^
[15:00:07] SMILES Parse Error: Failed parsing SMILES 'ZDQWESQEGGJUCH-UHFFFAOYSA-N' for input: 'ZDQWESQEGGJUCH-UHFFFAOYSA-N'
[15:00:07] SMILES Parse Error: syntax error while parsing: LTMQZVLXCLQPCT-UHFFFAOYSA-N
[15:00:07] SMILES Parse Error: check for mistakes around position 1:
[15:00:07] LTMQZVLXCLQPCT-UHFFFAOYSA-N
[15:00:07] ^
[15:00:0

Removidos 7 compostos com SMILES inválidos durante a preparação.
Quantidade de compostos após preparação dos SMILES: 5083
Padronizando os SMILES...


[15:00:07] Running Normalizer
[15:00:07] Running Uncharger
[15:00:07] Running Normalizer
[15:00:07] Running Uncharger
[15:00:07] Running Normalizer
[15:00:07] Running Uncharger
[15:00:07] Running Normalizer
[15:00:07] Running Uncharger
[15:00:07] Running Normalizer
[15:00:07] Running Uncharger
[15:00:07] Running Normalizer
[15:00:07] Running Uncharger
[15:00:07] Running Normalizer
[15:00:07] Running Uncharger
[15:00:07] Running Normalizer
[15:00:07] Running Uncharger
[15:00:07] Running Normalizer
[15:00:07] Running Uncharger
[15:00:07] Running Normalizer
[15:00:07] Running Uncharger
[15:00:07] Running Normalizer
[15:00:07] Running Uncharger
[15:00:07] Running Normalizer
[15:00:07] Running Uncharger
[15:00:07] Running Normalizer
[15:00:07] Running Uncharger
[15:00:07] Running Normalizer
[15:00:07] Running Uncharger
[15:00:07] Running Normalizer
[15:00:07] Running Uncharger
[15:00:07] Running Normalizer
[15:00:07] Running Uncharger
[15:00:07] Running Normalizer
[15:00:07] Running Uncharg

Removidos 0 compostos durante a padronização dos SMILES.
Quantidade de compostos após padronização dos SMILES: 5083
Removendo sais das moléculas...


[15:00:10] WARNING: not removing hydrogen atom without neighbors
[15:00:10] WARNING: not removing hydrogen atom without neighbors
[15:00:10] WARNING: not removing hydrogen atom without neighbors
[15:00:10] WARNING: not removing hydrogen atom without neighbors
[15:00:10] WARNING: not removing hydrogen atom without neighbors
[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted und

Removidos 107 compostos durante a remoção de sais.
Quantidade de compostos após remoção de sais: 4976
Salvando sais removidos em '/Users/francisco/Library/CloudStorage/OneDrive-Pessoal/Documentos/LabMol/SOFIA/MQ/Paper OdorSight/DATA/Curation/removed_salts.csv'...
Sais removidos salvos com sucesso.
Removendo misturas de moléculas...
Removidos 123 compostos que são misturas.
Quantidade de compostos após remoção de misturas: 4853
Salvando misturas removidas em '/Users/francisco/Library/CloudStorage/OneDrive-Pessoal/Documentos/LabMol/SOFIA/MQ/Paper OdorSight/DATA/Curation/removed_mixtures.csv'...
Misturas removidas salvas com sucesso.
Calculando InChI para as moléculas...


[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefi

Removidos 0 compostos devido a falhas no cálculo do InChI.
Quantidade de compostos após cálculo de InChI: 4853
Removendo duplicatas para tarefas de classificação...
Duplicatas concordantes removidas: 561
Duplicatas discordantes removidas: 33
Quantidade de compostos após remoção de duplicatas (classificação): 4259
Quantidade final de compostos: 4259
Salvando dataset curado em '/Users/francisco/Library/CloudStorage/OneDrive-Pessoal/Documentos/LabMol/SOFIA/MQ/Paper OdorSight/DATA/Curation/curated_dataset.csv'...
Dataset curado salvo com sucesso.
Salvando duplicatas concordantes removidas em '/Users/francisco/Library/CloudStorage/OneDrive-Pessoal/Documentos/LabMol/SOFIA/MQ/Paper OdorSight/DATA/Curation/removed_concordant_duplicates.csv'...
Duplicatas concordantes removidas salvas com sucesso.
Salvando duplicatas discordantes removidas em '/Users/francisco/Library/CloudStorage/OneDrive-Pessoal/Documentos/LabMol/SOFIA/MQ/Paper OdorSight/DATA/Curation/removed_discordant_duplicates.csv'...
Dup

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefined stereo

[15:00:10] WARNING: Omitted undefi